In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
import os
import matplotlib.pyplot as plt
import keras
print(tf.__version__)
from keras.layers import Activation, Dense
print(keras.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
(X_train, y_train) , (X_test, y_test) = fashion_mnist.load_data()
plt.imshow(X_train[0])


print("stats about training data")
print(X_train.shape)
print(y_train.shape)

print("stats about test data")
print(X_test.shape)
print(y_test.shape)

print('A single training data and its label')
print('Output Label',y_train[0])
print('Raw input\n',X_train[0])

In [ ]:
# Scale the input test and training data.
X_train, X_test = X_train.astype('float32') / 255. , X_test.astype('float32') / 255.

In [ ]:
print('Scaled version')
print(X_train[0])

In [ ]:
plt.imshow(X_train[0])

In [ ]:
# Convert from a 2D matrix to a single column vector
X_train = X_train.reshape(X_train.shape[0], 28 * 28)
X_test  = X_test.reshape(X_test.shape[0], 28 * 28)
print(X_train.shape)
print(X_test.shape)

In [ ]:
# Create a directory to store the model
if not os.path.exists('models'):
    os.makedirs('models')

In [ ]:
train_set = (X_train, y_train)
test_set  = (X_test,y_test)

In [ ]:
tf.random.set_seed(42)
mnist_fashion_model = tf.keras.Sequential()
mnist_fashion_model.add(Dense(10, input_shape = (784,), activation='sigmoid'))
print(mnist_fashion_model.summary())

In [ ]:
# Model tuning parameters
tune_epochs = 10
tune_batch_size = 6000
tune_learning_rate = 0.1
mnist_fashion_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=tune_learning_rate),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             metrics=['accuracy'],
             )
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('/models/fashion_mnist_final.h5',monitor='val_accuracy',
                                                        save_best_only=True,mode='max')
history = mnist_fashion_model.fit(train_set[0], train_set[1], batch_size = tune_batch_size, epochs=tune_epochs,validation_data=test_set,
                   callbacks=[checkpoint_callback])
'''
print(history.params)
print(history.history.keys)
print(history.history['loss'])
print(history.history['val_loss'])
'''

In [ ]:
plt.figure(figsize=[15,5])
plt.plot(history.history['loss'], 'g')
plt.plot(history.history['val_loss'], 'b')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend(['Training', 'Validation'], loc='upper right');
plt.grid(True)

In [ ]:
# Check the performance of the model
assert mnist_fashion_model.evaluate(X_test,y_test, verbose=1)[1] >= 0.85

In [ ]:
# Make predictions
class_names = ['T-shirts/top', 'Trouser','Pullover', 'Dress', 'Coat',
              'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']
num_rows = 5
num_cols = 5
num_samples = num_rows * num_cols
sample_index = np.random.randint(0, 10000, num_samples)

X_sample = X_test[sample_index]
gt_sample = y_test[sample_index]
plt.figure(figsize=(24,18))
for i , sample in enumerate(X_sample):
    gt_class = class_names[gt_sample[i]]
    sample = sample.reshape(1, 784)
    pred = mnist_fashion_model.predict(sample)
    pred_class_number = np.argmax(pred)
    pred_class = class_names[pred_class_number]
    if gt_class == pred_class:
        print("Correct prediction")
        print("Actual class is:", gt_class, "Predicted class:", pred_class)
    else:
        print("In-Correct prediction")
        print("Actual class is:", gt_class, "Predicted class:", pred_class)